In [1]:
import os
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import json
import datetime

class ImageAnnotator:
    def __init__(self, root, image_dir):
        self.root = root
        self.root.title("Image Annotator")
        self.image_dir = image_dir
        self.annotations = {}
        
        
        self.annotations_file = os.path.join(image_dir, "annotations.json")
        if os.path.exists(self.annotations_file):
            with open(self.annotations_file, 'r') as f:
                self.annotations = json.load(f)
        
        
        self.image_files = [f for f in os.listdir(image_dir) 
                          if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        self.current_index = 0
        
        
        self.setup_gui()
        self.load_current_image()
    
    def setup_gui(self):
        
        self.main_frame = ttk.Frame(self.root, padding="10")
        self.main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        
        self.image_label = ttk.Label(self.main_frame)
        self.image_label.grid(row=0, column=0, columnspan=2, pady=10)
        
        
        ttk.Label(self.main_frame, text="Label:").grid(row=1, column=0, sticky=tk.W)
        self.label_entry = ttk.Entry(self.main_frame, width=40)
        self.label_entry.grid(row=1, column=1, pady=5)
        
        ttk.Label(self.main_frame, text="Notes:").grid(row=2, column=0, sticky=tk.W)
        self.notes_text = tk.Text(self.main_frame, height=4, width=40)
        self.notes_text.grid(row=2, column=1, pady=5)
        
        
        button_frame = ttk.Frame(self.main_frame)
        button_frame.grid(row=3, column=0, columnspan=2, pady=10)
        
        ttk.Button(button_frame, text="Previous", command=self.prev_image).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Save", command=self.save_annotation).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Next", command=self.next_image).pack(side=tk.LEFT, padx=5)
        
        
        self.progress_var = tk.StringVar()
        ttk.Label(self.main_frame, textvariable=self.progress_var).grid(row=4, column=0, columnspan=2)
    
    def load_current_image(self):
        if 0 <= self.current_index < len(self.image_files):
            image_path = os.path.join(self.image_dir, self.image_files[self.current_index])
            image = Image.open(image_path)
            
            # Resize image to fit display (max 800x600)
            image.thumbnail((800, 600))
            photo = ImageTk.PhotoImage(image)
            
            self.image_label.configure(image=photo)
            self.image_label.image = photo
            
           
            self.progress_var.set(f"Image {self.current_index + 1} of {len(self.image_files)}")
            
            
            current_file = self.image_files[self.current_index]
            if current_file in self.annotations:
                self.label_entry.delete(0, tk.END)
                self.label_entry.insert(0, self.annotations[current_file].get('label', ''))
                self.notes_text.delete('1.0', tk.END)
                self.notes_text.insert('1.0', self.annotations[current_file].get('notes', ''))
            else:
                self.label_entry.delete(0, tk.END)
                self.notes_text.delete('1.0', tk.END)
    
    def save_annotation(self):
        if self.current_index < len(self.image_files):
            current_file = self.image_files[self.current_index]
            self.annotations[current_file] = {
                'label': self.label_entry.get(),
                'notes': self.notes_text.get('1.0', tk.END).strip(),
                'timestamp': datetime.datetime.now().isoformat()
            }
            
            with open(self.annotations_file, 'w') as f:
                json.dump(self.annotations, f, indent=4)
    
    def next_image(self):
        if self.current_index < len(self.image_files) - 1:
            self.save_annotation()
            self.current_index += 1
            self.load_current_image()
    
    def prev_image(self):
        if self.current_index > 0:
            self.save_annotation()
            self.current_index -= 1
            self.load_current_image()


if __name__ == "__main__":
    image_dir = r"C:\Users\eKasi_SWT_COM00862\Downloads\doctor\doctor"  
    root = tk.Tk()
    app = ImageAnnotator(root, image_dir)
    root.mainloop()